<a href="https://colab.research.google.com/github/sneha-venkat/BDSProject/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
templates = [
              ["[prop1] of [nns]","SELECT [prop1] FROM [nns]"],
              ["[agg] [prop1] for each [breakdown]","SELECT [agg]([prop1]) , [breakdown] FROM [prop1] GROUP BY [breakdown]"],
              ["[prop1] of [nns] by [breakdown]","SELECT [prop1] , [breakdown] FROM [nns] GROUP BY [breakdown]"],
              ["[prop1] of [nns] in [location] by [breakdown]","SELECT [prop1] , [breakdown] FROM [nns] WHERE location = '[location]' GROUP BY [breakdown]"],
              ["[nns] having [prop1] between [number1] and [number2]","SELECT name FROM [nns] WHERE [prop1] > [number1] and [prop1] < [number2]"],
              ["[prop] by [breakdown]","SELECT name , [breakdown] FROM [prop] GROUP BY [breakdown]"],
              ["[agg] of [prop1] of [nn]","SELECT [agg]([prop1]) FROM [nn]"],
              ["[prop1] of [nns] before [year]","SELECT [prop1] FROM [nns] WHERE date < [year]"],
              ["[prop1] of [nns] after [year] in [location]","SELECT [prop1] FROM [nns] WHERE date > [year] AND location='[location]'"],
              ["[nns] [verb] after [year] in [location]","SELECT name FROM [nns] WHERE location = '[location]' AND date > [year]"],
              ["[nns] having [prop1] between [number1] and [number2] by [breakdown]","SELECT name , [breakdown] FROM [nns] WHERE [prop1] < [number1] AND [prop1] > [number2] GROUP BY [breakdown]"],
              ["[nns] with a [prop1] of maximum [number1] by their [breakdown]","SELECT name , [breakdown] FROM [nns] WHERE [prop1] <= [number1] GROUP BY [breakdown]"],
              ["[prop1] and [prop2] of [nns] since [year]","SELECT [prop1] , [prop2] FROM [nns] WHERE date > [year]"],
              ["[nns] which have both [prop1] and [prop2]","SELECT name FROM [nns] WHERE [prop1] IS true AND [prop2] IS true"],
              ["Top [number1] [nns] by [prop1]","SELECT name FROM [nns] ORDER BY [prop1] DESC LIMIT [number1]"]
]
template = random.choice(templates)
print("Sample Query Template  :", template[0])
print("SQL Translation        :", template[1])

Sample Query Template  : [nns] with a [prop1] of maximum [number1] by their [breakdown]
SQL Translation        : SELECT name , [breakdown] FROM [nns] WHERE [prop1] <= [number1] GROUP BY [breakdown]


In [5]:
%pip install layer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.1/343.1 KB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 121.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6

In [15]:
import layer
from layer.decorators import *

In [16]:

objects = ["countries","wines","wineries","tasters", "provinces","grapes","cities","bottles","deliveries"]
object_single = ["country","wine","winery","taster", "province","grape","city","bottle", "delivery"]
properties = ["points","price","taste","title","texture","age","duration","acidity","flavor","level"]
aggs = [["average","avg"], ["total","sum"],["count","count"], ["minimum","min"], ["maximum","max"]]
breakdowns = ["quality","price","province","country","point", "variety","flavor","age"]
locations = ["Italy","US","Portugal","Spain","Chile","Turkey","Canada"]
verbs = ["produced","bottled"]

regex = r"\[([a-z0-9]*)\]"
number_of_samples = 2500

@dataset("english_sql_translations")
def build_dataset():
    rows = []
    for index in range(0,number_of_samples):
        template = random.choice(templates)
        nl = template[0]
        sql = template[1]

        matches = re.finditer(regex, nl, re.MULTILINE)

        for matchNum, match in enumerate(matches, start=1):
            key = match.group()
            prop = None
            prop_sql = None
            if key.startswith("[prop"):
                prop = random.choice(properties)
                prop_sql = prop.replace(" ","_").lower()
            if key in ["[nns]"]:
                prop = random.choice(objects)
                prop_sql = prop
            if key in ["[nn]"]:
                prop = random.choice(object_single)
                prop_sql = prop.replace(" ","_").lower()
            if key == "[breakdown]":
                prop = random.choice(breakdowns)
                prop_sql = prop.replace(" ","_").lower()
            if key == "[verb]":
                prop = random.choice(verbs)
                prop_sql = prop.replace(" ","_").lower()
            if key == "[agg]":
                aggregation = random.choice(aggs)
                prop = aggregation[0]
                prop_sql = aggregation[1]
            if key == "[location]":
                prop = random.choice(locations)
                prop_sql = prop
            if key.startswith("[number"):
                prop = str(random.randint(1,1000))
                prop_sql = prop
            if key.startswith("[year"):
                prop = str(random.randint(1950,2022))
                prop_sql = prop
            

            if prop is not None:
                nl = nl.replace(key,prop)
                sql = sql.replace(key,prop_sql)
        
        prefix = random.randint(1,20)
        if prefix == 1:
            nl = "Show me "+nl
        elif prefix == 2:
            nl = "List "+nl
        elif prefix == 3:
            nl = "List of "+nl
        elif prefix == 4:
            nl = "Find "+nl
        rows.append([nl,sql])

    df = pd.DataFrame(rows, columns=["query", "sql"])
    return df

In [9]:
from torch.utils.data import Dataset
class EnglishToSQLDataSet(Dataset):

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.target_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

    self.data["query"] = "translate English to SQL: "+self.data["query"]
    self.data["sql"] = "<pad>" + self.data["sql"] + "</s>"

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.target_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [10]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    import torch

    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]

        step = (epoch * len(loader)) + _
        layer.log({"loss": float(loss)}, step)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [17]:
@model("t5-tokenizer")
@fabric("f-medium")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_tokenizer():
    from transformers import T5Tokenizer
    # Load tokenizer from Hugging face
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    return tokenizer

@model("t5-english-to-sql")
@fabric("f-gpu-small")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_model():
    from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
    from transformers import T5Tokenizer, T5ForConditionalGeneration
    import torch.nn.functional as F
    from torch import cuda
    import torch
    
    parameters={
        "BATCH_SIZE":8,          
        "EPOCHS":3,              
        "LEARNING_RATE":2e-05,          
        "MAX_SOURCE_TEXT_LENGTH":75,   
        "MAX_TARGET_TEXT_LENGTH":75,
        "SEED": 42
    }

    # Log parameters to Layer
    layer.log(parameters)
    
    # Set seeds for reproducibility
    torch.manual_seed(parameters["SEED"])
    np.random.seed(parameters["SEED"])
    torch.backends.cudnn.deterministic = True

    # Load tokenizer from Layer
    tokenizer = layer.get_model("t5-tokenizer").get_train()

    # Load pretrained model from Hugging face
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    device = 'cuda' if cuda.is_available() else 'cpu'
    model.to(device)

    dataframe = layer.get_dataset("english_sql_translations").to_pandas()
    source_text = "query"
    target_text = "sql"

    dataframe = dataframe[[source_text,target_text]]

    train_dataset = dataframe.sample(frac=0.8,random_state = parameters["SEED"])
    train_dataset = train_dataset.reset_index(drop=True)

    layer.log({"FULL Dataset": str(dataframe.shape),
             "TRAIN Dataset": str(train_dataset.shape)
             })

    training_set = EnglishToSQLDataSet(train_dataset, tokenizer, parameters["MAX_SOURCE_TEXT_LENGTH"], parameters["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)

    dataloader_paramaters = {
      'batch_size': parameters["BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }

    training_loader = DataLoader(training_set, **dataloader_paramaters)
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=parameters["LEARNING_RATE"])

    for epoch in range(parameters["EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    return model

In [13]:
%pip install sentencepiece==0.1.96

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00


In [25]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00


In [26]:
# model = layer.get_model('layer/t5-fine-tuning-with-layer/models/t5-english-to-sql').get_train()
# tokenizer = layer.get_model('layer/t5-fine-tuning-with-layer/models/t5-tokenizer').get_train()
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = tokenizer = T5Tokenizer.from_pretrained("t5-small")


/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [18]:
def greet(query):
    input_ids = tokenizer.encode(f"translate English to SQL: {query}", return_tensors="pt")
    outputs = model.generate(input_ids, max_length=1024)
    sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql

In [27]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1248: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [28]:
greet("Show me the average price of wines in Italy by provinces")

'SELECT AVG Price ( USD ) FROM table WHERE Country = italy AND Province = provinces'

In [29]:
greet("Cars built after 2020 and manufactured in Italy")

'SELECT Cars FROM table WHERE Year built > 2020 AND Manufacturer = italy'

In [30]:
greet("Top 10 cities by their population")

'SELECT Top 10 Cities FROM table WHERE Population = 20,000'

In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("dbernsohn/t5_wikisql_en2SQL")

model = AutoModelForSeq2SeqLM.from_pretrained("dbernsohn/t5_wikisql_en2SQL")

In [33]:
greet("Show me the average price of wines in Italy by provinces")

'SELECT AVG Price FROM table WHERE Province = italy'

In [34]:
greet("Cars built after 2020 and manufactured in Italy")

'SELECT Cars FROM table WHERE Year built > 2020 AND Manufacturer = italy'

In [35]:
greet("Top 10 cities by their population")

'SELECT Top 10 Cities FROM table'

In [36]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [37]:
greet("Show me the average price of wines in Italy by provinces")

'Zeig mir den Durchschnittspreis für Weine in Italien nach Provinzen'

In [38]:
greet("Cars built after 2020 and manufactured in Italy")

'SQL-Autos nach 2020 und in Italien hergestellt'

In [39]:
greet("Top 10 cities by their population")

'Top 10 Städte nach ihrer Bevölkerung'

In [41]:
tokenizer = layer.get_model("layer/t5-fine-tuning-with-layer/models/t5-tokenizer")

LayerClientServiceUnavailableException: ignored

In [42]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("vvn/Text_to_SQL_BART_spider-three-ep")

model = AutoModelForSeq2SeqLM.from_pretrained("vvn/Text_to_SQL_BART_spider-three-ep")

In [43]:
greet("Show me the average price of wines in Italy by provinces")

'SELECT avg(Price),  province FROM WINE WHERE country  =  \'Italy\' ORDER BY T1.language_translates_in_SELECT T2.translate_English FROM SELECT AS T1 JOIN APPELLATIONS AS T2 ON T1_APPELLATION_IN_T2.APPEILATION_ISTENCT = T3.apt_client_id WHERE T2_language_code =  "English"'

In [44]:
greet("Cars built after 2020 and manufactured in Italy")

"SELECT T1.translate_English FROM Cars AS T1 JOIN Manufacturers AS T2 ON T2.manufacturer_id  =  T3.code JOIN Vehicles AS T3 ON T4.Vehicle_ID  = T5.Manufacturer_ID WHERE T1 LIKE '%i%' AND T2 LIKE 'Italy%' UNION SELECT T2 FROM Cars WHERE T3 LIKE '2020%'"

In [ ]:
greet("Top 10 cities by their population")

In [45]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/tapex-large-sql-execution")

model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/tapex-large-sql-execution")

In [47]:
greet("Show me the average price of wines in Italy by provinces")

AttributeError: ignored

In [48]:
greet("Cars built after 2020 and manufactured in Italy")

AttributeError: ignored

In [ ]:
greet("Top 10 cities by their population")